In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library("factoextra")
library("maptree")
library("ggpubr")
library("ica")

library(repr)
fileDir = "/home/beraslan/jovian-work/analysisSingle/"
geneCutOff=4
targetCutOff=15
nFactors=15



In [ ]:
myICAFactors = data.frame(read.csv("./../TextFiles/ICA_15_factors_effectedGenes_paper.csv",
                                   stringsAsFactors=FALSE), 
                          stringsAsFactors=FALSE)
myICAFactors_guides = myICAFactors[,grep("guide", colnames(myICAFactors))]
myICAFactors_genes = myICAFactors[,grep("gene", colnames(myICAFactors))]

selGuides = unique(unlist(myICAFactors_guides))
selGuides = selGuides[selGuides != ' ']

In [ ]:
selCoefs <- read.csv("./../TextFiles/SignificantCoefMatrix.csv", row.names = 1)
selCoefs <- t(selCoefs)
KOGenes <-  colnames(selCoefs)
KOGenes <- sapply(KOGenes, function(x){strsplit(x,"_")[[1]][2]})
colnames(selCoefs) <- KOGenes

In [ ]:
guideModulesN <- data.frame(read.csv("./../TextFiles/ME_GuideModules_leiden_6_Modules.csv"),  row.names = 1)
rownames(guideModulesN) <- guideModulesN$GuideName 
guideModulesN$GuideGroup <- factor(guideModulesN$GuideGroup)
guideModulesN$GuideGroup <- paste0("K", guideModulesN$GuideGroup)
guideModulesN$GuideColor <- NULL

head(guideModulesN)

In [ ]:
k <- icaimax(selCoefs, nc = 15, center = T)
myH <- k$S


In [ ]:
facR2Mat = data.frame(matrix(0, nrow = 16, ncol = ncol(selCoefs)))
rownames(facR2Mat) = c(paste0("Factor_",1:15),"ALL")
colnames(facR2Mat) = colnames(selCoefs)

for(fac in 1:ncol(myH)){
    for(pert in 1:ncol(selCoefs)){
        myDF = data.frame(x=myH[,fac], y=selCoefs[,pert])
        k = lm("y ~ x", data= myDF)
        myPredict = predict(k, newdata = myDF)

        myR2 = computeR2(myDF$y, myPredict)
        
        facR2Mat[paste0("Factor_", fac), colnames(selCoefs)[pert] ] = myR2
    }
}

myDF = copy(data.frame(myH))

for(pert in 1:ncol(selCoefs)){
    myDF$y = selCoefs[,pert]
  
    k = lm("y ~ .", data= myDF)
    myPredict = predict(k, newdata = myDF)

    myR2 = computeR2(myDF$y, myPredict)
        
    facR2Mat["ALL", colnames(selCoefs)[pert] ] = myR2

}

In [ ]:
facR2Mat = facR2Mat[,order(-facR2Mat["ALL",])]

In [ ]:
allExp = facR2Mat["ALL",]
allExp

In [ ]:
write.csv(facR2Mat, "ExplainedVariancePerFactor.csv")

In [ ]:
highExp = names(allExpVar[allExpVar>0.25])

In [ ]:
annoCols<-list(GeneGroup=c(G0='#A6CEE3',
                           G1='#1F78B4',
                           G2='#B2DF8A',
                           G3='#33A02C', 
                           G4='#FB9A99', 
                           G5='#FDBF6F', 
                           G6='#FF7F00', 
                           G7='#CAB2D6',
                           G8='#6A3D9A', 
                           G9='#FFFF99', 
                           G10="#B5651D" ),
                 GuideGroup=c(K0="#1f77b4",
                              K1="#ff7f0e",
                              K2="#279e68",
                              K3="#d62728", 
                              K4="#aa40fc", 
                              K5="#8c564b"))

In [ ]:
# options(repr.plot.width=28, repr.plot.height=6)
# #facR2MatPlot = copy(data.frame(facR2Mat[,highExp]))
# facR2MatPlot = copy(data.frame(facR2Mat[,highExp]))
# facR2MatPlot[facR2MatPlot > 0.2] = 0.2
# facR2MatPlot[facR2MatPlot < 0.05] = 0.05

# kk = pheatmap((facR2MatPlot[1:15,]),
#          cluster_cols = TRUE,
#          cluster_rows = FALSE,
#          clustering_method="ward.D2",
#          clustering_distance_rows="euclidean", colorRampPalette(c("white", "orange", "red"))(100))

# facR2MatPlot = facR2MatPlot[,kk$tree_col$order]

# annotDFcol <- guideModulesN[colnames(facR2MatPlot),]
# annotDFcol$GuideName = NULL
# head(annotDFcol)

# pheatmap(facR2MatPlot,
#          cluster_cols = TRUE,
#          cluster_rows = FALSE,
#          annotation_col = annotDFcol,
#          annotation_colors = annoCols,
#          clustering_method="ward.D2",
#          clustering_distance_rows="euclidean", colorRampPalette(c("white", "orange", "red"))(100))



In [ ]:
options(repr.plot.width=30, repr.plot.height=6)
facR2MatPlot = copy(facR2Mat[,highExp])
facR2MatPlot[facR2MatPlot > 0.2] = 0.2
facR2MatPlot[facR2MatPlot < 0.05] = 0.05

annotDFcol <- guideModulesN[colnames(facR2MatPlot),]
annotDFcol$GuideName = NULL
head(annotDFcol)

xx  = pheatmap(facR2MatPlot[1:15,],
         cluster_cols = TRUE,
         cluster_rows = FALSE,
         annotation_col = annotDFcol,
         annotation_colors = annoCols,
         clustering_method="ward.D2",
         fontsize_row=20,
         clustering_distance_rows="euclidean", colorRampPalette(c("white", "orange", "red"))(100))

save_pheatmap_pdf(xx, "Figure_4A.pdf", width=28, height=6)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

selGenes = c("Rbx1", "Rfwd2", "Cul4b", "Det1", "Ddb1" )
facR2MatPlot = copy(facR2Mat[,selGenes])
facR2MatPlot[facR2MatPlot > 0.4] = 0.4
facR2MatPlot[facR2MatPlot < 0.05] = 0.05

annotDFcol <- guideModulesN[colnames(facR2MatPlot),]
annotDFcol$GuideName = NULL
head(annotDFcol)

xx  = pheatmap(facR2MatPlot[1:15,],
         cluster_cols = TRUE,
         cluster_rows = FALSE,
         annotation_col = annotDFcol,
         annotation_colors = annoCols,
         clustering_method="ward.D2",
         fontsize=20,
         clustering_distance_rows="euclidean", colorRampPalette(c("white", "orange", "red"))(100))


In [ ]:
facR2MatPlot = copy(facR2Mat[,highExp])
facR2MatPlot = facR2MatPlot[,xx$tree_col$order]

facR2MatPlot[facR2MatPlot > 0.8] = 0.8

facR2MatPlot[facR2MatPlot < 0.2] = 0.2

annotDFcol <- guideModulesN[colnames(facR2MatPlot),]
annotDFcol$GuideName = NULL
head(annotDFcol)

kk = pheatmap(facR2MatPlot,
         cluster_cols = FALSE,
         cluster_rows = FALSE,
         annotation_col = annotDFcol,
         annotation_colors = annoCols,
         clustering_distance_rows="euclidean", colorRampPalette(c("white", "orange", "red"))(100))

save_pheatmap_pdf(kk, "Figure_4A_2.pdf", width=28, height=6)
